# Ejemplo básico de K-Means en Python utilizando `sklearn`

En este notebook se presenta un ejemplo básico de K-Means en Python utilizando la librería `sklearn` (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html). Como es habitual, empezamos importando las librerías necesarias:

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

Creamos el DataFrame pandas con los datos de entrada: 8 muestras (filas) y 2 variables (columnas):

In [ ]:
d = {
    'A': [0.93, 0.91, 0.89, 0.87, 0.19, 0.17, 0.23, 0.21], 
    'B': [0.09, 0.07, 0.13, 0.11, 0.83, 0.81, 0.79, 0.77]
}

df = pd.DataFrame(data=d)

df

Representamos gráficamente la matriz para observar la naturaleza de los datos:

In [ ]:
df.plot.scatter(x = 'A', y = 'B', c = 'black')
plt.show()

En este caso parece que hay dos grupos bien diferenciados, así que vamos a utilizar la función `KMeans` para agrupar las muestras en 2 clusters:

In [ ]:
kmeans = KMeans(n_clusters=2).fit(df)

kmeans

Obtenemos las coordenadas de los dos centroides:

In [ ]:
centroids = kmeans.cluster_centers_

print('Centroide del primer cluster:', str(centroids[0,:]))
print('Centroide del segundo cluster:', str(centroids[1,:]))

Y creamos nuevamente el gráfico anterior añadiendo estos dos puntos:

In [ ]:
df.plot.scatter(x = 'A', y = 'B', c = 'black')
plt.scatter(centroids[:, 0], centroids[:, 1], c = 'red', marker = 'x')
plt.show()

Obtenemos las asignaciones de las muestras a los clusters y las añadimos a nuestro dataset:

In [ ]:
labels = kmeans.predict(df)

df['group'] = labels

df

Y utilizamos las etiquetas para dar un color a los puntos:

In [ ]:
colors = np.where(df['group']==0,'m','g')

df.plot.scatter(x = 'A', y = 'B', c = colors)
plt.scatter(centroids[:, 0], centroids[:, 1], c = 'red', marker = 'x')
plt.show()

Podemos comprobar cómo la media de los puntos asignados a cada cluster se corresponde con el centroide:

In [ ]:
centroid0 = df[df['group'] == 0].sum(axis = 0) / 4
centroid1 = df[df['group'] == 1].sum(axis = 0) / 4

print('Centroide del primer cluster:\n', centroid0)
print('\nCentroide del segundo cluster:\n', centroid1)

Aplicamos el método *elbow* a este ejemplo, utilizando de 1 a 3 clusters:

In [ ]:
k_range = range(1, 4)

kmeans = [KMeans(n_clusters=i) for i in k_range]
inertia = [kmeans[i].fit(df).inertia_ for i in range(len(kmeans))]

plt.xlabel('k')
plt.ylabel('inertia')
plt.plot(k_range, inertia, linestyle='--', marker='o', color='black')
plt.show()

Finalmente, haremos el análisis de silhouette para este caso. Para ello, sklearn incluye dos funciones: `silhouette_samples`, que devuelve los scores de todas las muestras dadas (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_samples.html), y `silhouette_score`, que devuelve el score medio sobre todas las muestras (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html). En la documentación de sklearn se puede encontrar un ejemplo de este tipo de análisis: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html

Dado que esta métrica requiere de al menos dos grupos (clusters), vamos a calcular los silhouette scores solo para k = 2 y k = 3:

In [ ]:
from sklearn import metrics

labels_k = [kmeans[i].predict(df) for i in range(1, 3)]
silhouettes = [metrics.silhouette_score(df, labels_k[i-1], metric = 'euclidean') for i in range(1, 3)]

print('silhouette para k = 2:', silhouettes[0])
print('silhouette para k = 3:', silhouettes[1])

In [ ]:
df_silhouette = df.copy()

df_silhouette['group (k = 2)'] = labels_k[0]
df_silhouette['silhouette (k = 2)'] = metrics.silhouette_samples(df, labels_k[0], metric = 'euclidean')
df_silhouette['group (k = 3)'] = labels_k[1]
df_silhouette['silhouette (k = 3)'] = metrics.silhouette_samples(df, labels_k[1], metric = 'euclidean')

df_silhouette

## Escalado de variables

Tal y como comentamos en clase de teoría, en ciertos casos puede ser necesario aplicar un escalado de variables para conseguir mejores resultados y evitar que ciertas variables dominen el método empleado. Comenzamos por añadir una nueva variable C al ejemplo anterior y aplicar el algoritmo K-Means con k = 2. Tal y como se puede ver, esta variable toma valores de 0.56 o 0.55 independientemente del "grupo" al que pertenece según el K-Means anterior, y el resultado de este nuevo K-Means incluyendo dicha variable es el mismo.

In [ ]:
d_1 = {
    'A': [0.93, 0.91, 0.89, 0.87, 0.19, 0.17, 0.23, 0.21], 
    'B': [0.09, 0.07, 0.13, 0.11, 0.83, 0.81, 0.79, 0.77],
    'C': [0.55, 0.56, 0.55, 0.56, 0.55, 0.56, 0.55, 0.56]
}

df_1 = pd.DataFrame(data=d_1)

kmeans_1 = KMeans(n_clusters=2).fit(df_1)

df_1['group'] = kmeans_1.predict(df_1)

df_1

Ahora, creamos un DataFrame similar pero con la variable C en una escala superior, es decir, con los valores 550 y 560 distribuidos de igual modo entre las 8 muestras. En este caso, se puede observar como estos valores tan altos dominan las distancias y se obtiene un agrupamiento diferente.

In [ ]:
d_2 = {
    'A': [0.93, 0.91, 0.89, 0.87, 0.19, 0.17, 0.23, 0.21], 
    'B': [0.09, 0.07, 0.13, 0.11, 0.83, 0.81, 0.79, 0.77],
    'C': [550, 560, 550, 560, 550, 560, 550, 560]
}

df_2 = pd.DataFrame(data=d_2)

kmeans_2 = KMeans(n_clusters=2).fit(df_2)

df_2['group'] = kmeans_2.predict(df_2)

df_2

Para evitar que ciertas variables dominen el cálculo de los grupos, es habitual aplicar un escalado de variables que las sitúe en escalas comparables. La técnica más habitual es la estandarización, que consiste en escalar cada variable restando a cada valor la media y dividiéndolo por la desviación típica. De esta manera, cada variable tendrá media = 0 y desviación típica = 1. Aplicamos el escalado utilizando el `StandardScaler` de sklearn y comprobamos el resultado del agrupamiento, que vuelve a dejar de estar influenciado por la variable C.

In [ ]:
d_2 = {
    'A': [0.93, 0.91, 0.89, 0.87, 0.19, 0.17, 0.23, 0.21], 
    'B': [0.09, 0.07, 0.13, 0.11, 0.83, 0.81, 0.79, 0.77],
    'C': [550, 560, 550, 560, 550, 560, 550, 560]
}
df_2 = pd.DataFrame(data=d_2)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_2_scaled = pd.DataFrame(scaler.fit_transform(df_2))

print(df_2_scaled)

print(f'Valor medio de cada columna: {scaler.mean_}')

kmeans_2 = KMeans(n_clusters=2).fit(df_2_scaled)

df_2_scaled['group'] = kmeans_2.predict(df_2_scaled)

df_2_scaled